Teledetección y Python para analizar los efectos de la COVID-19

La reducción de la mobilidad y la actividad económica durante la pandemia de la COVID-19 ha provocado también una reducción significativa de las emisiones de NO2 a la atmósfera. Utilizando técnicas de teledetección y algunas librerías en Python, podemos analizar los efectos de la COVID-19 en base a la reducción de estos gases contaminantes.

Datos de partida
El satélite Sentinel-5p de la Agencia Espacial Europea lleva a bordo el sensor TROPOMI. Este sensor permite monitorizar concentraciones de gases como el ozono, el metano, el monóxido de carbono, el dióxido de nitrógeno o el óxido de azufre.

En esta ocasión nos centraremos en el dióxido de nitrógeno (NO2). Se trata de un gas generado fundamentalmente por la actividad humana, como resultado de la combustión de los vehículos y determinados procesos industriales.

Con una resolución espacial de unos 7kmx7km, Sentinel-5p cubre diariamente toda la superfície terrestre. De este modo puede hacerse un seguimiento exhaustivo de la evolución de estos gases, y analizar episodios muy concretos como el producido durante la pandemia.

A través de esta aplicación (https://maps.s5p-pal.com/) desarrollada por la ESA y en base a los datos de Sentinel-5p puede visualizarse como evoluciona la concentración de NO2 en todo el planeta.

![NO2 concentration](https://www.unigis.es/wp-content/uploads/2020/07/sentinel5-p_no2.png)

En esta ocasión utilizaremos la API de Copernicus Open Acces Hub junto con la librería Python Sentinelsat para obtener las imágenes que necesitamos.

Tal y como ya hemos comentado, para la búsqueda y obtención de las imágenes utilizaremos la librería Sentinelsat. El procesado y la visualización lo haremos utilizando las librerías HARP y VISAN respectivamente. Ambas forman parte de Atmospheric Toolbox, un paquete de librerías para facilitar a científicos el consumo, procesado y análisis de datos atmosféricos obtenidos con técnicas de teledetección.

In [4]:
!pip install sentinelsat

  Using cached sentinelsat-0.14-py2.py3-none-any.whl (36 kB)


Descarga de datos con Sentinelsat
Sentinelsat es una librería que facilita la búsqueda, filtrado y descarga de las imágenes del programa Copernicus disponibles en el repositorio de Copernicus Open Acces Hub. La conexión con el repositorio de imágenes Sentinel 5p a partir de la cuenta genérica se hará del siguiente modo:

In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

In [2]:
from datetime import date, datetime, timedelta

In [8]:
api = SentinelAPI('s5pguest', 's5pguest', 'https://s5phub.copernicus.eu/dhus')

In [5]:
# indicamos el bbox de la zona de descarga
footprint = geojson_to_wkt(read_geojson('../data/bbox_spain.geojson'))

In [6]:
footprint = geojson_to_wkt(read_geojson('../data/bbox_london.geojson'))

In [9]:
products = api.query(footprint,
                      date=(date(2020, 2, 2), date(2020, 5, 8)),
                      producttype='L2__NO2___',
                      platformname='Sentinel-5')

Querying products:  29%|##8       | 100/350 [00:00<?, ?product/s]

In [11]:
def cambiar_extension():
    """
    Función para cambiar la extensión de las escenas
    (de .zip a .nc)
    """

    for filename in os.listdir(os.path.dirname(os.path.abspath(__file__))):
        base_file, ext = os.path.splitext(filename)
        if ext == ".zip":
            os.rename(filename, base_file + ".nc")

In [13]:
def mosaico_escenas(nombre_nc):
    """
    Función para crear un mosaico de productos
    """

    # Buscamos en el directorio los productos S5P

    file_names = []
    products = [] # array con los productos a juntar

    print("Importar productos a HARP")


    i = 0
    for filename in os.listdir(os.path.dirname(os.path.abspath(__file__))):
        base_file, ext = os.path.splitext(filename)

        if ext == ".nc" and base_file.split('_')[0] == 'S5P':
            product_name = base_file + "_" + str(i)

            try:
                product_name = harp.import_product(base_file + ext,
                                                    operations="latitude > -55 [degree_north]; latitude < 80 [degree_north]; tropospheric_NO2_column_number_density_validity > 75;bin_spatial(231,-55,0.5,721,-180,0.5)",
                                                    post_operations="bin();squash(time, (latitude,longitude))")
                print("Producto " + base_file + ext + " importado")

                products.append(product_name)
            except:
                print ("Producto no importado")

            i = i + 1


    try:
        print("Ejecutando operación de unión")
        product_bin = harp.execute_operations(products, "", "bin()")
        #harp.export_product(product_bin, file_names[0].split('_')[8]+".nc")

        print("Exportar producto unido")
        harp.export_product(product_bin, str(nombre_nc)+".nc")

    except:
        print("No se ha podido realizar la unión")


    # Borramos los archivos originales, para no ocupar tanta memoria



    for filename in os.listdir(os.path.dirname(os.path.abspath(__file__))):
        base_file, ext = os.path.splitext(filename)

        if ext == ".nc" and base_file.split('_')[0] == 'S5P':
            os.remove(base_file + ext)


In [14]:
def descarga_datos(grupo_dias, parametros_acceso, bbox):
    """Función para la descarga de datos.
    Keyword arguments:
    grupo_dias -- se hará la media del grupo de dias definido
    parametros_acceso -- credenciales y url acceso al repositorio de imágenes
    bbox -- área de busqueda de las imagenes
    """


    dia_inicio = date(2019, 12, 23)  # fecha inicio
    dia_final = date(2020, 6, 29)   # fecha final

    numero_grupo = 1

    while dia_inicio <= dia_final:

        # se realizará una búsqueda por área, fecha y producto
        products = parametros_acceso.query(bbox,
                             date=(dia_inicio, dia_inicio + timedelta(grupo_dias)),
                             producttype='L2__NO2___',
                             platformname='Sentinel-5')

        # mostramos por pantalla los productos encontrados
        #print(products)

        dia_inicio = dia_inicio + timedelta(grupo_dias)

        try:
            # iniciamos la descarga de los productos encontrados
            api.download_all(products)

            # una vez descargadas las imagenes, renombramos los archivos
            cambiar_extension()

            # realizamos el mosaico de escenas
            mosaico_escenas(dia_inicio - timedelta(grupo_dias))
            numero_grupo = numero_grupo + 1

        except:
            print("Error en la descarga y/o ejecución del proceso")


# llamamos la función para la descarga de datos
# descarga_datos(7, api, footprint)